In [1]:
import os
os.environ["CUDA_VISIBLE_DEVICES"] = "0,1"
os.environ["TOKENIZERS_PARALLELISM"] = "false"

In [2]:
import warnings
warnings.filterwarnings("ignore")

In [3]:
import numpy as np
import pandas as pd
import os
from tqdm import tqdm
import bitsandbytes as bnb
import torch
import torch.nn as nn
import transformers
from datasets import Dataset
from peft import LoraConfig, PeftConfig
from trl import SFTTrainer
from trl import setup_chat_format
from transformers import (AutoModelForCausalLM, 
                          AutoTokenizer, 
                          BitsAndBytesConfig, 
                          TrainingArguments, 
                          pipeline, 
                          logging)
from sklearn.metrics import (accuracy_score, 
                             classification_report, 
                             confusion_matrix)
from sklearn.model_selection import train_test_split

In [4]:
print(f"pytorch version {torch.__version__}")

pytorch version 2.1.2+cu121


In [5]:
filename = "/home/user10/code/datamining/output/translated_reviews_41600.csv"
df = pd.read_csv(filename,encoding="utf-8", encoding_errors="replace")
print(df.head(3))
df['prompt'] = df['text'].apply(
    lambda row: f"""
                Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative".

                [{row}] = """.strip()
               )


  sentiment                                               text  \
0  positive                  I got it well before my due date.   
1  positive  Thank you lannister stores I loved to buy thro...   
2  positive  I recommend it. I think it's effective. On the...   

                          review_id                          order_id  \
0  e64fb393e7b32834bb789ff8bb30750e  658677c97b385a9be170737859d3511b   
1  f7c4243c7fe1938f181bec41a392bdeb  8e6bfb81e283fa7e4f11123a3fb894f1   
2  8670d52e15e00043ae7de4c01cc2fe06  b9bf720beb4ab3728760088589c62129   

   review_score review_comment_title  \
0             5                  NaN   
1             5                  NaN   
2             4            recomendo   

                              review_comment_message review_creation_date  \
0              Recebi bem antes do prazo estipulado.  2017-04-21 00:00:00   
1  Parabéns lojas lannister adorei comprar pela I...  2018-03-01 00:00:00   
2  aparelho eficiente. no site a marca do aparelh.

In [6]:
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
device="balanced"
print(f"working on {device}")

working on balanced


In [7]:
from peft import AutoPeftModelForCausalLM

finetuned_model = "./trained_weigths/"
compute_dtype = getattr(torch, "float16")
tokenizer = AutoTokenizer.from_pretrained("/home/user10/code/datamining/input")

model = AutoPeftModelForCausalLM.from_pretrained(
     finetuned_model,
     torch_dtype=compute_dtype,
     return_dict=False,
     low_cpu_mem_usage=True,
     device_map="balanced",
)

merged_model = model.merge_and_unload()
# merged_model.save_pretrained("./merged_model",safe_serialization=True, max_shard_size="2GB")
# tokenizer.save_pretrained("./merged_model")

Loading checkpoint shards: 100%|██████████| 2/2 [00:12<00:00,  6.32s/it]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:

def save_df(texts,df):
    os.system("clear")
    n=len(texts)
    print(len(texts))
    new_df=df[:len(texts)]
    new_df['predict_sentiment']=texts
    result_df = new_df[['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
    print(result_df.head(3))
    result_df.to_csv(f'output/predicted_reviews_{len(texts)}.csv', index=False)

    print(f"处理完成，新文件已保存为 output/predicted_reviews_{len(texts)}.csv")
    print(result_df.columns,len(result_df))
def predict(model, tokenizer):
    y_pred = []
    for i in tqdm(range(len(df))):
        prompt = df.iloc[i]["prompt"]
        pipe = pipeline(task="text-generation", 
                        model=model, 
                        tokenizer=tokenizer, 
                        max_new_tokens = 1, 
                        temperature = 0.0,
                       )
        result = pipe(prompt)
        answer = result[0]['generated_text'].split("=")[-1]
        if "positive" in answer:
            y_pred.append("positive")
        elif "negative" in answer:
            y_pred.append("negative")
        elif "neutral" in answer:
            y_pred.append("neutral")
        else:
            y_pred.append("none")
        if (i!=0 and i%500==0) or (i==(len(df)-1)):
            print(result[0]['generated_text'],y_pred[-1],df.iloc[i])
        if (i!=0 and i%5000==0) or (i==(len(df)-1)):
            save_df(y_pred,df)
    return y_pred

In [11]:
with torch.no_grad():
    y_pred = predict(merged_model, tokenizer)


  1%|          | 503/41600 [00:32<42:21, 16.17it/s] 

Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative".

                [Product with defects presents risk for being unreliable, I've already e-mailed and e-mailed and had no feedback.] = negative negative sentiment                                                           negative
text                       Product with defects presents risk for being u...
review_id                                   3395e28d6009c3c18dd879ae600d91bc
order_id                                    1e7666fef7090957578fd3a4b58cf9ba
review_score                                                               1
review_comment_title                                                     NaN
review_comment_message     Produto com defeito apresenta risco por não se...
review_creation_date                                  

  2%|▏         | 1003/41600 [01:02<41:06, 16.46it/s]

Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative".

                [It arrived in perfect condition and on time. Thank you very much for your time.] = positive positive sentiment                                                           positive
text                       It arrived in perfect condition and on time. T...
review_id                                   6f64dec3ab5e638053c1a1d6e2c7e204
order_id                                    4f39a9f31c5a7d46ecfc92571700a6f9
review_score                                                               5
review_comment_title                                                     NaN
review_comment_message     Chegou em perfeito estado e antes do prazo pre...
review_creation_date                                     2017-07-21 00:00:00
review_ans

  4%|▎         | 1503/41600 [01:33<41:24, 16.14it/s]

...

Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative".

                [Good quality, beautiful product.] = neutral neutral sentiment                                                           positive
text                                        Good quality, beautiful product.
review_id                                   c24682eabda0197608fdccb153283f2f
order_id                                    2a46a0636b60369577c40749497b77a4
review_score                                                               5
review_comment_title                                                     NaN
review_comment_message                    Produto de boa qualidade e bonito.
review_creation_date                                     2018-03-20 00:00:00
review_answer_timestamp                                  20

100%|█████████▉| 41599/41600 [42:32<00:00, 16.35it/s]

Analyze the sentiment of the news headline enclosed in square brackets, 
                determine if it is positive, neutral, or negative, and return the answer as 
                the corresponding sentiment label "positive" or "neutral" or "negative".

                [Very fast. I liked the ease to buy fast delivery.] = positive positive sentiment                                                           positive
text                       Very fast. I liked the ease to buy fast delivery.
review_id                                   1ebb9e6fd55f241efb7861e58f9d8e70
order_id                                    afed00b9cae687f1f1a07bad548aaa50
review_score                                                               5
review_comment_title                                            Muito rápido
review_comment_message     Amei a facilidade para comprar a rapidez na en...
review_creation_date                                     2018-07-07 00:00:00
review_answer_timestamp                 

100%|██████████| 41600/41600 [42:33<00:00, 16.29it/s]

处理完成，新文件已保存为 output/predicted_reviews_41600.csv
Index(['sentiment', 'predict_sentiment', 'text', 'review_id', 'order_id',
       'review_score', 'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp'],
      dtype='object') 41600


In [13]:
os.system("clear")
n=len(y_pred)
print(len(y_pred))
new_df=df[:len(y_pred)]
new_df['predict_sentiment']=y_pred
#result_df = new_df[['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
# print(new_df.head(3))
# result_df.to_csv(f'output/predicted_reviews_{len(texts)}.csv', index=False)

# print(f"处理完成，新文件已保存为 output/predicted_reviews_{len(texts)}.csv")
print(new_df.columns,len(new_df))

41600
Index(['sentiment', 'text', 'review_id', 'order_id', 'review_score',
       'review_comment_title', 'review_comment_message',
       'review_creation_date', 'review_answer_timestamp', 'prompt',
       'predict_sentiment'],
      dtype='object') 41600


In [20]:
low_reviews_df=new_df[(new_df['sentiment']==new_df['predict_sentiment']) | (new_df['predict_sentiment']=='neutral') | (new_df['sentiment']=='neutral')][['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
high_reviews_df=new_df[new_df['sentiment']==new_df['predict_sentiment']][['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]

In [21]:
print(len(low_reviews_df))
low_reviews_df.head(1)

41021


,sentiment,predict_sentiment,text,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,positive,neutral,I got it well before my due date.,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06


In [22]:
print(len(high_reviews_df))
high_reviews_df.head(1)

19521


,sentiment,predict_sentiment,text,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
4,positive,positive,"Great recommendation. Reliable customer, good ...",3948b09f7c818e2d86c9a546758b2335,e51478e7e277a83743b6f9991dbfa3fb,5,Super recomendo,"Vendedor confiável, produto ok e entrega antes...",2018-05-23 00:00:00,2018-05-24 03:00:01


In [23]:
not_low_reviews_df=new_df[~((new_df['sentiment']==new_df['predict_sentiment']) | (new_df['predict_sentiment']=='neutral') | (new_df['sentiment']=='neutral'))][['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]
not_high_reviews_df=new_df[~(new_df['sentiment']==new_df['predict_sentiment'])][['sentiment','predict_sentiment', 'text','review_id', 'order_id', 'review_score', 'review_comment_title','review_comment_message', 'review_creation_date','review_answer_timestamp']]

In [25]:
print(len(not_low_reviews_df))
not_low_reviews_df.head(4)

579


,sentiment,predict_sentiment,text,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
68,positive,negative,If it was glass it broke; it came in the box w...,14613df954694f243dedeaa2c8a3a5c2,1e91914c9706a5d4665b8c5a6294b107,4,4,Se fosse vidro tinha quebrado; veio na caixa ...,2018-05-15 00:00:00,2018-05-16 23:41:51
188,positive,negative,It's a poor product. I have no question about ...,6d7d0a5e33c92adf782b137720eeefa6,d9aefa5d4ccd369c783f5121e069a470,5,produto fraco,quanto ao site não tenho o que questionar! sem...,2018-05-11 00:00:00,2018-05-14 12:04:52
189,positive,negative,Bombs are everywhere.,5f8ed42b9eee8adbd7344caca4ca507e,05ad071b550b1cf2815e8bf3bec3291f,4,NaN,bom em todos os aspectos,2017-05-20 00:00:00,2017-05-23 10:43:03
245,positive,none,Note 10.,31b60e2f8f2acfe8f213ca252f1066c1,7dc5018fbc594b1ce9a945e6bcfc83c3,5,NaN,nota 10,2018-04-25 00:00:00,2018-04-28 17:14:18


In [26]:
print(len(not_high_reviews_df))
not_high_reviews_df.head(4)

22079


,sentiment,predict_sentiment,text,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
0,positive,neutral,I got it well before my due date.,e64fb393e7b32834bb789ff8bb30750e,658677c97b385a9be170737859d3511b,5,NaN,Recebi bem antes do prazo estipulado.,2017-04-21 00:00:00,2017-04-21 22:02:06
1,positive,neutral,Thank you lannister stores I loved to buy thro...,f7c4243c7fe1938f181bec41a392bdeb,8e6bfb81e283fa7e4f11123a3fb894f1,5,NaN,Parabéns lojas lannister adorei comprar pela I...,2018-03-01 00:00:00,2018-03-02 10:26:53
2,positive,neutral,I recommend it. I think it's effective. On the...,8670d52e15e00043ae7de4c01cc2fe06,b9bf720beb4ab3728760088589c62129,4,recomendo,aparelho eficiente. no site a marca do aparelh...,2018-05-22 00:00:00,2018-05-23 16:45:47
3,positive,neutral,But a little bit of labor for that good value.,4b49719c8a200003f700d3d986ea1a19,9d6f15f95d01e79bd1349cc208361f09,4,NaN,"Mas um pouco ,travando...pelo valor ta Boa.\r\n",2018-02-16 00:00:00,2018-02-20 10:52:22


In [29]:
not_high_reviews_df.to_csv(f'output/not_high_reviews.csv', index=False)
not_low_reviews_df.to_csv(f'output/not_low_reviews.csv', index=False)
high_reviews_df.to_csv(f'output/high_reviews.csv', index=False)
low_reviews_df.to_csv(f'output/low_reviews.csv', index=False)

579


,sentiment,predict_sentiment,text,review_id,order_id,review_score,review_comment_title,review_comment_message,review_creation_date,review_answer_timestamp
68,positive,negative,If it was glass it broke; it came in the box w...,14613df954694f243dedeaa2c8a3a5c2,1e91914c9706a5d4665b8c5a6294b107,4,4,Se fosse vidro tinha quebrado; veio na caixa ...,2018-05-15 00:00:00,2018-05-16 23:41:51
188,positive,negative,It's a poor product. I have no question about ...,6d7d0a5e33c92adf782b137720eeefa6,d9aefa5d4ccd369c783f5121e069a470,5,produto fraco,quanto ao site não tenho o que questionar! sem...,2018-05-11 00:00:00,2018-05-14 12:04:52
189,positive,negative,Bombs are everywhere.,5f8ed42b9eee8adbd7344caca4ca507e,05ad071b550b1cf2815e8bf3bec3291f,4,NaN,bom em todos os aspectos,2017-05-20 00:00:00,2017-05-23 10:43:03
245,positive,none,Note 10.,31b60e2f8f2acfe8f213ca252f1066c1,7dc5018fbc594b1ce9a945e6bcfc83c3,5,NaN,nota 10,2018-04-25 00:00:00,2018-04-28 17:14:18
